## Prepocessing and lenght Analysis
Analisi della lunghezza dei discorsi per decidere i politici da utilizzare per l'analisi

In [19]:
import pandas as pd
import logging
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')
import time
import pprint
import pickle
import random
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from pprint import pprint
from datetime import datetime
import re
import json
import pandas
import operator
from functools import reduce
import editdistance
import copy
import regex
from nltk.tokenize import RegexpTokenizer
from string import punctuation
from nltk.corpus import stopwords
import textrazor
import spacy

In [6]:
import random

In [20]:
nlp = spacy.load("it_core_news_sm")
stopwords = set(stopwords.words('italian'))

### Preprocesing

In [13]:
cognomi = {}
cognomi['I'] = pickle.load( open( "Discorsi-I-legis/cognomi_discorsi.p", "rb" ) )
cognomi['XII'] = pickle.load( open( "Discorsi-XII-legis/cognomi_discorsi.p", "rb" ) )
cognomi['XVII'] = pickle.load( open( "Discorsi-XVII-legis/cognomi_discorsi.p", "rb" ) )
cognomi['XVIII'] = pickle.load( open( "Discorsi-XVIII-legis/cognomi_discorsi.p", "rb" ) )

In [14]:
legis = {l:{} for l in cognomi.keys()}
for l in legis.keys():
    legis[l] = {cog: pickle.load( open( "Discorsi-"+l+"-legis/"+cog+".p", "rb" ) ) for cog in cognomi[l]}

In [78]:
# numero di discorsi
for l in legis.keys():
    print(l+" "+str(len(legis[l])))

I 30
XII 80
XVII 175
XVIII 108


In [82]:
len(legis['I']['MORO'])

1648

In [79]:
30+80+175+108

393

In [83]:
# media numero di discorsi prima del preprocessing
n_disco = {l:[] for l in legis}
for l in legis:
    n_disco[l] = [len(legis[l][cog]) for cog in legis[l].keys()]

In [84]:
for l in n_disco:
    print(l+" "+str(np.mean(n_disco[l])))
    print(l+" "+str(np.median(n_disco[l])))

I 1235.4333333333334
I 367.0
XII 351.9
XII 109.5
XVII 559.0
XVII 179.0
XVIII 300.3425925925926
XVIII 83.0


### Salvataggio discorsi puliti ###

In [ ]:
tokens = lambda text: [x.lemma_ for x in nlp(text) if x.pos_ not in ['PUNCT', 'SPACE'] and not x.is_stop]
#all_tokens = lambda text: [x.lemma_ for x in nlp(text) if x.pos_ not in ['PUNCT', 'SPACE']]

In [ ]:
separator = ' '
cont = 0
leg = 'I'
for cog in legis[leg].keys():
    if legis[leg][cog]!= []:
        cont = 0
        for l in legis[leg][cog]:
            l = legis[leg][cog][cont]         
            l = separator.join(l).lower()
            l = tokens(l)
            l = list(l)
            legis[leg][cog][cont] = l
            cont = cont+1

In [ ]:
for cog in legis[leg].keys():
    pickle.dump( legis[leg][cog], open( 'Discorsi_puliti_I/'+cog+".p", "wb" ) )

In [ ]:
separator = ' '
cont = 0
leg = 'XVII'
for cog in legis[leg].keys():
    if legis[leg][cog]!= []:
        cont = 0
        for l in legis[leg][cog]:
            l = legis[leg][cog][cont]         
            l = separator.join(l).lower()
            l = tokens(l)
            l = list(l)
            legis[leg][cog][cont] = l
            cont = cont+1

In [ ]:
for cog in legis[leg].keys():
    pickle.dump( legis[leg][cog], open( 'Discorsi_puliti_XVII/'+cog+".p", "wb" ) )

In [ ]:
separator = ' '
cont = 0
leg = 'XII'
for cog in legis[leg].keys():
    if legis[leg][cog]!= []:
        cont = 0
        for l in legis[leg][cog]:
            l = legis[leg][cog][cont]         
            l = separator.join(l).lower()
            l = tokens(l)
            l = list(l)
            legis[leg][cog][cont] = l
            cont = cont+1

In [ ]:
for cog in legis[leg].keys():
    pickle.dump( legis[leg][cog], open( 'Discorsi_puliti_XII/'+cog+".p", "wb" ) )

In [ ]:
#XVIII legis in quanto ha i nomi dei partiti a inizio frase
leg = 'XVIII'
separator = ' '
cont = 0
for cog in legis[leg].keys():
    if legis[leg][cog]!= []:
        cont = 0
        for l in legis[leg][cog]:
            if l!= [] and l[0][0] == '(' and len(l[0])<15: #rimuovo i nomi dei partiti prima degli interventi
                    pprint(l[0])
                    legis[leg][cog][cont].remove(l[0])
            l = legis[leg][cog][cont]         
            l = separator.join(l).lower()
            l = tokens(l)
            l = list(l)
            legis[leg][cog][cont] = l
            cont = cont+1

In [ ]:
for cog in legis[leg].keys():
    pickle.dump( legis[leg][cog], open( 'Discorsi_puliti_XVIII/'+cog+".p", "wb" ) )

### Filtro
Per avere un dataset bilanciato e per problemi dovuti all'uso di TextRazor (max 500 discorsi), decido di prendere random 1000 discorsi per politico.

In [22]:
all_legis = {}
all_legis['I'] = {}
all_legis['XII'] = {}
all_legis['XVII'] = {} 
all_legis['XVIII'] =  {}

for cog in cognomi['I']:
    all_legis['I'][cog] = pickle.load( open( 'Discorsi_puliti_I/'+cog+".p", "rb" ) )
    
for cog in cognomi['XII']:
    all_legis['XII'][cog] = pickle.load( open( 'Discorsi_puliti_XII/'+cog+".p", "rb" ) )
    
for cog in cognomi['XVIII']:
    all_legis['XVIII'][cog] = pickle.load( open( 'Discorsi_puliti_XVIII/'+cog+".p", "rb" ) )
    
for cog in cognomi['XVII']:
    all_legis['XVII'][cog] = pickle.load( open( 'Discorsi_puliti_XVII/'+cog+".p", "rb" ) )

In [23]:
for l in all_legis:
    print(l)
    for cog in all_legis[l]:
        if len(all_legis[l][cog])>300:
            print(cog+" "+str(len(all_legis[l][cog])))

I
MORO 1648
DE GASPERI 563
FANFANI 814
GRONCHI 16892
SEGNI 1331
LEONE 3917
TOGLIATTI 350
DI VITTORIO 1456
GIOLITTI 883
BASSO 413
MARTINO 4668
LA MALFA 418
PACCIARDI 415
ALMIRANTE 680
RUSSO PEREZ 582
COVELLI 384
XII
IRENE PIVETTI 3141
LUCIANO VIOLANTE 5907
VITTORIO DOTTI 849
IGNAZIO LA RUSSA 2483
LORENZO ACQUARONE 3274
RAFFAELE DELLA \ALLE 3243
FRANCESCO STORACE 404
XVII
LAURA BOLDRINI 12302
ROBERTO GIACHETTI 15421
MARINA SERENI 11318
LUIGI DI MAIO 9666
SIMONE BALDELLI 10412
RESOCONTO STENOGRAFICO 663
IGNAZIO LA RUSSA 768
ROCCO BUTTIGLIONE 439
NICOLA MOLTENI 557
EMANUELE FIANO 493
GIANLUCA PINI 489
ARTURO SCOTTO 351
DAVIDE CAPARINI 328
FABIO RAMPELLI 363
DANILO TONINELLI 444
EDMONDO CIRIELLI 337
GIAN LUIGI GIGLI 413
MATTEO BRAGANTINI 389
PAOLA BINETTI 395
ETTORE ROSATO 388
CARLO SIBILIA 432
MASSIMILIANO FEDRIGA 650
ROCCO PALESE 754
STEFANO ALLASIA 329
VITTORIO FERRARESI 413
STEFANO QUARANTA 408
ARCANGELO SANNICANDRO 619
CRISTIAN INVERNIZZI 323
FRANCESCO PAOLO SISTO 716
ANDREA COLLETTI 5

In [24]:
# rimuovo i discorsi composti da meno di 10 parole
discorsi_ridotti = {l: {} for l in all_legis.keys()}

for l in all_legis.keys():
    for cog in all_legis[l].keys():
        discorsi_ridotti[l][cog] = [d for d in all_legis[l][cog] if len(d)>10]            

In [25]:
leng = {}

for l in discorsi_ridotti.keys():
    print(l)
    leng[l] = []
    for cog in discorsi_ridotti[l]:
        if len(discorsi_ridotti[l][cog]) > 300:
            print(cog+" "+str(len(discorsi_ridotti[l][cog])))
            leng[l].append((cog, len(discorsi_ridotti[l][cog])))

I
MORO 1452
DE GASPERI 373
FANFANI 512
GRONCHI 9138
SEGNI 860
LEONE 2204
DI VITTORIO 688
GIOLITTI 622
BASSO 332
MARTINO 2711
ALMIRANTE 480
XII
IRENE PIVETTI 1537
LUCIANO VIOLANTE 3420
VITTORIO DOTTI 494
IGNAZIO LA RUSSA 1456
LORENZO ACQUARONE 1921
RAFFAELE DELLA \ALLE 1823
XVII
LAURA BOLDRINI 4387
ROBERTO GIACHETTI 8347
MARINA SERENI 5297
LUIGI DI MAIO 4485
SIMONE BALDELLI 5871
RESOCONTO STENOGRAFICO 578
ROCCO BUTTIGLIONE 364
NICOLA MOLTENI 413
GIANLUCA PINI 353
FABIO RAMPELLI 304
PAOLA BINETTI 364
MASSIMILIANO FEDRIGA 415
ROCCO PALESE 587
ARCANGELO SANNICANDRO 470
FRANCESCO PAOLO SISTO 536
ANDREA COLLETTI 388
ALFONSO BONAFEDE 305
DAVIDE CRIPPA 491
XVIII
ROBERTO FICO 1810
MARIA ROSARIA CARFAGNA 1852
MARIA EDERA SPADONI 1409
ETTORE ROSATO 2333
FABIO RAMPELLI 2036
FEDERICO FORNARO 332


In [28]:
for l in leng.keys():
    leng[l] = sorted(leng[l], key=lambda tup: tup[1], reverse = True)

In [29]:
cognomi_analisi = {}
cognomi_analisi['I'] = [leng['I'][i][0] for i in range(0,5)]
cognomi_analisi['XII'] = [leng['XII'][i][0] for i in range(0,5)]
cognomi_analisi['XVII'] = [leng['XVII'][i][0] for i in range(0,5)]
cognomi_analisi['XVIII'] = [leng['XVIII'][i][0] for i in range(0,5)]

In [30]:
cognomi_analisi

{'I': ['GRONCHI', 'MARTINO', 'LEONE', 'MORO', 'SEGNI'],
 'XII': ['LUCIANO VIOLANTE',
  'LORENZO ACQUARONE',
  'RAFFAELE DELLA \\ALLE',
  'IRENE PIVETTI',
  'IGNAZIO LA RUSSA'],
 'XVII': ['ROBERTO GIACHETTI',
  'SIMONE BALDELLI',
  'MARINA SERENI',
  'LUIGI DI MAIO',
  'LAURA BOLDRINI'],
 'XVIII': ['ETTORE ROSATO',
  'FABIO RAMPELLI',
  'MARIA ROSARIA CARFAGNA',
  'ROBERTO FICO',
  'MARIA EDERA SPADONI']}

In [ ]:
pickle.dump(cognomi_analisi, open( "cognomi_analisi.p", "wb" ) )

<b> Salvataggio gruppi 1000 </b>

In [33]:
discorsi_analisi = {}
for l in cognomi_analisi.keys():
    discorsi_analisi[l] = {}
    for cog in cognomi_analisi[l]:
        if len(discorsi_ridotti[l][cog]) > 1000:
            discorsi_analisi[l][cog] = []
            pos = random.sample(range(0, len(discorsi_ridotti[l][cog])), 1000)
            for p in pos:
                discorsi_analisi[l][cog].append(discorsi_ridotti[l][cog][p])
        else:
            discorsi_analisi[l][cog] = discorsi_ridotti[l][cog]

In [ ]:
for l in discorsi_analisi.keys():
    pickle.dump(discorsi_analisi[l], open( l+"_discorsi_analisi.p", "wb" ) )

### Long discorsi analisi ###
Salvo per ogni legis i primi 2 politici aventi il maggior numero di discorsi, senza fare preprocessing. Questo serve per poter creare il modello word2vec.

In [38]:
cognomi_analisi_long = {}
for l in cognomi_analisi.keys():
    cognomi_analisi_long[l] = [cognomi_analisi[l][i] for i in range(0,2)]

In [39]:
cognomi_analisi_long

{'I': ['GRONCHI', 'MARTINO'],
 'XII': ['LUCIANO VIOLANTE', 'LORENZO ACQUARONE'],
 'XVII': ['ROBERTO GIACHETTI', 'SIMONE BALDELLI'],
 'XVIII': ['ETTORE ROSATO', 'FABIO RAMPELLI']}

In [40]:
# rimuovo i discorsi composti da meno di 10 parole
discorsi_ridotti_long = {}

for l in cognomi_analisi_long.keys():
    for cog in cognomi_analisi_long[l]:
           discorsi_ridotti_long[cog] = [d for d in legis[l][cog] if len(d)>10] #uso i discorsi originali, non quelli preprocessati

In [ ]:
# metto solo il lower case e rimuovo nomi partiti a inizio frase
separator = ' '
cont = 0

for cog in discorsi_ridotti_long.keys():
    if discorsi_ridotti_long[cog]!= []:
            cont = 0
            for l in discorsi_ridotti_long[cog]:
                if l!= [] and l[0][0] == '(' and len(l[0])<15: #rimuovo i nomi dei partiti prima degli interventi
                        pprint(l[0])
                        discorsi_ridotti_long[cog][cont].remove(l[0])

                disc = discorsi_ridotti_long[cog][cont]         
                disc = separator.join(disc).lower()
                reg = re.sub("[^A-Za-z']+", ' ', disc)
                final = reg.split(' ')
                discorsi_ridotti_long[cog][cont] = final
                cont = cont+1
    print(cog)

In [45]:
for cog in discorsi_ridotti_long.keys():
    print(len(discorsi_ridotti_long[cog]))

11099
3294
4412
2445
11659
8014
3466
3319


In [ ]:
pickle.dump(discorsi_ridotti_long, open("Discorsi_analisi/discorsi_analisi_long.p", "wb" ) )

### Salvataggio discorsi per Bigram e Trigram ###
Mi servono i discorsi per analisi bigram e trigram, dove non vengano rimosse le stopwords.

In [67]:
discorsi_analisi_ngram = {}

for l in legis.keys():
    for cog in cognomi_analisi[l]:
        discorsi_analisi_ngram[cog] = legis[l][cog]

In [68]:
[(cog,len(discorsi_analisi_ngram[cog])) for cog in discorsi_analisi_ngram.keys()]

[('GRONCHI', 16892),
 ('MARTINO', 4668),
 ('LEONE', 3917),
 ('MORO', 1648),
 ('SEGNI', 1331),
 ('LUCIANO VIOLANTE', 5907),
 ('LORENZO ACQUARONE', 3274),
 ('RAFFAELE DELLA \\ALLE', 3243),
 ('IRENE PIVETTI', 3141),
 ('IGNAZIO LA RUSSA', 2483),
 ('ROBERTO GIACHETTI', 15421),
 ('SIMONE BALDELLI', 10412),
 ('MARINA SERENI', 11318),
 ('LUIGI DI MAIO', 9666),
 ('LAURA BOLDRINI', 12302),
 ('ETTORE ROSATO', 4667),
 ('FABIO RAMPELLI', 4157),
 ('MARIA ROSARIA CARFAGNA', 3860),
 ('ROBERTO FICO', 5296),
 ('MARIA EDERA SPADONI', 3240)]

In [70]:
print(discorsi_analisi_ngram['IRENE PIVETTI'][11])

['L', "'ordine", 'del', 'giorno', 'reca', 'la', 'deliberazione', 'ai', 'sensi', 'dell', "'articolo", '96', '-bis,', 'comma', '3', 'del', 'regolamento', 'sul', 'disegno', 'di', 'legge', 'Conversione', 'in', 'legge', 'del', 'decreto', '-leg-ge', '29', 'aprile', '1994', 'n', '262', 'recante', 'provvedi', '-menti', 'finalizzati', 'alla', 'razionalizzazione', 'del', "-l'indebitamento", 'delle', 'società', 'per', 'azioni', 'interamente', 'possedute', 'dallo', 'Stato', 'Ricordo', 'che', 'nella', 'seduta', 'del', '31', 'maggio', 'scorso', 'la', 'I', 'Commissione', '(Affari', 'costituzio', '-nali)', 'ha', 'espresso', 'parere', 'contrario', 'sull', "'esi-stenza", 'dei', 'presupposti', 'richiesti', 'dal', 'secondo', 'comma', 'dell', "'articolo", '77', 'della', 'Costituzione', 'per', 'l', "'adozione", 'del', 'decreto', '-legge', 'n', '262', 'del', '1994', 'di', 'cui', 'al', 'disegno', 'di', 'legge', 'di', 'conversio', '-ne', 'n', '401', 'Ha', 'facoltà', 'di', 'parlare', 'il', 'presidente', 'della'

In [ ]:
separator = ' '
cont = 0

for cog in discorsi_analisi_ngram.keys():
    if discorsi_analisi_ngram[cog]!= []: 
            cont = 0
            for l in discorsi_analisi_ngram[cog]:
                if l!= [] and l[0][0] == '(' and len(l[0])<15: #rimuovo i nomi dei partiti prima degli interventi
                        pprint(l[0])
                        discorsi_analisi_ngram[cog][cont].remove(l[0])

                disc = discorsi_analisi_ngram[cog][cont]         
                disc = separator.join(disc).lower()
                reg = re.sub("[^A-Za-z']+", ' ', disc)
                final = reg.split(' ')
                discorsi_analisi_ngram[cog][cont] = final
                cont = cont+1
    print(cog)

In [ ]:
pickle.dump(discorsi_analisi, open("Discorsi_analisi/discorsi_ngram.p",'wb'))